# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Cities Data Output File (CSV)
output_data_file = "/Users/ashleypatricia/Documents/GitHub/python-api-challenge/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [8]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

605

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).


In [14]:
api_url = "https://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

query_url = f"{api_url}units={units}&appid={weather_api_key}"
#print(query_url)

# Create empty list 
final_data = []
pro_records = 1
sets = 1

print("Beginning Data Retrieval")
print("-----------------------------")

for index, city in enumerate(cities): 
    # Creating sets of 50 
    if index >= 50 and index % 50 == 0:
        sets += 1
        pro_records = 1

    # Create url for each city
    city_url = query_url + "&q=" + city.replace(" ","+")
    print(f"Processing Record {pro_records} of Set {sets} | {city}")
    pro_records += 1

    try: 
        response = requests.get(city_url).json()    
        lat = response["coord"]["lat"]
        lng = response["coord"]["lon"]
        max_temp = response["main"]["temp_max"]
        humidity = response["main"]["humidity"]
        cloudiness = response["clouds"]["all"]
        windspeed = response["wind"]["speed"]
        country = response["sys"]["country"]
        pull_date = response["dt"]
        final_data.append({"City": city, 
            "Lat": lat, 
            "Lng": lng, 
            "Max Temp": max_temp, 
            "Humidity": humidity, 
            "Cloudiness": cloudiness, 
            "Wind Speed": windspeed, 
            "Country": country, 
            "Date": pull_date})

    except (KeyError, IndexError):
        print("City *not* found. Skipping...")
        pass

print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")
    

| upernavik
Processing Record 5 of Set 4 | kyra
Processing Record 6 of Set 4 | nizhneyansk
City *not* found. Skipping...
Processing Record 7 of Set 4 | omsukchan
Processing Record 8 of Set 4 | san carlos
Processing Record 9 of Set 4 | berlevag
Processing Record 10 of Set 4 | kununurra
Processing Record 11 of Set 4 | mahibadhoo
Processing Record 12 of Set 4 | port hardy
Processing Record 13 of Set 4 | forbes
Processing Record 14 of Set 4 | bubaque
Processing Record 15 of Set 4 | hobyo
Processing Record 16 of Set 4 | vygonichi
Processing Record 17 of Set 4 | naze
Processing Record 18 of Set 4 | mehamn
Processing Record 19 of Set 4 | altay
Processing Record 20 of Set 4 | tiksi
Processing Record 21 of Set 4 | carnarvon
Processing Record 22 of Set 4 | adeje
Processing Record 23 of Set 4 | cape town
Processing Record 24 of Set 4 | geraldton
Processing Record 25 of Set 4 | cherskiy
Processing Record 26 of Set 4 | sioux lookout
Processing Record 27 of Set 4 | atuona
Processing Record 28 of Set

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [17]:
# See new dataframe
city_df = pd.DataFrame(final_data)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,port elizabeth,-33.9180,25.5701,66.20,94,90,10.36,ZA,1619043503
1,bluff,-46.6000,168.3333,51.01,98,93,3.00,NZ,1619043504
2,punta arenas,-53.1500,-70.9167,42.80,81,0,2.37,CL,1619043250
3,jamestown,42.0970,-79.2353,32.00,93,90,16.11,US,1619043216
4,tuktoyaktuk,69.4541,-133.0374,12.20,79,40,26.46,CA,1619043505


In [ ]:
# Export to csv
city_df.to_csv("/Users/ashleypatricia/Documents/GitHub/python-api-challenge/WeatherPy/city data.csv", index = False)

In [19]:
# Display the dataframe
city_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,5.510000e+02
mean,18.881249,21.032202,60.118548,71.174229,55.994555,7.852559,1.619044e+09
std,33.285361,88.913013,20.494534,22.010711,37.746306,5.396548,1.204089e+02
min,-54.800000,-175.200000,-4.650000,4.000000,0.000000,0.110000,1.619043e+09
25%,-8.164600,-58.633450,45.940000,61.000000,20.000000,3.710000,1.619044e+09
50%,21.283300,24.662200,64.400000,77.000000,68.000000,6.600000,1.619044e+09
75%,47.249650,102.389850,77.335000,87.500000,93.000000,11.330000,1.619044e+09
max,78.218600,179.316700,93.630000,100.000000,100.000000,32.210000,1.619044e+09


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [21]:
#  Get the indices of cities that have humidity over 100%.
high_hum = city_df[city_df["Humidity"] > 100]
high_hum

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
'''
x_values = 
plt.scatter(x_values, data, marker="o", facecolors="red", edgecolors="black", s=x_values, alpha=.75)
plt.xlabel = "Latitude"
plt.ylabel = "Max Temperature (F)"
plt.title = "City Latitude vs Max Temperature"

'''

## Latitude vs. Humidity Plot

In [ ]:
'''
x_values = 
plt.scatter(x_values, data, marker="o", facecolors="red", edgecolors="black", s=x_values, alpha=.75)
plt.xlabel = "Latitude"
plt.ylabel = "Humidity(%)"
plt.title = "City Latitude vs Humidity"
'''

## Latitude vs. Cloudiness Plot

In [ ]:
'''
x_values = 
plt.scatter(x_values, data, marker="o", facecolors="red", edgecolors="black", s=x_values, alpha=.75)
plt.xlabel = "Latitude"
plt.ylabel = "Cloudiness (%)"
plt.title = "City Latitude vs Cloudiness"
'''

## Latitude vs. Wind Speed Plot

In [ ]:
'''
x_values = 
plt.scatter(x_values, data, marker="o", facecolors="red", edgecolors="black", s=x_values, alpha=.75)
plt.xlabel = "Latitude"
plt.ylabel = "Wind Speed (mph"
plt.title = "City Latitude vs Wind Speed"
'''

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
'''
x_values = 
y_values = 
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel("Latitude")
plt.ylabel("Max Temp")
plt.show

'''

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
'''
x_values = 
y_values = 
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel("Latitude")
plt.ylabel("Max Temp")
plt.show

'''

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
'''
x_values = 
y_values = 
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.show

'''

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
'''
x_values = 
y_values = 
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.show

'''

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
'''
x_values = 
y_values = 
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.show

'''

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
'''
x_values = 
y_values = 
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.show

'''

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
'''
x_values = 
y_values = 
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.show

'''

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
'''
x_values = 
y_values = 
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.show

'''